In [ ]:
import matplotlib.pyplot as plt
import cv2
import numpy as np
import pandas as pd

def print_cell(image, title, index):
    plt.subplot(1,10,index)
    plt.imshow(image)
    plt.title(title)
    plt.axis('off')
    
def build_image_names(image_id: str) -> list:
    # mt is the mitchondria
    mt = f'/kaggle/input/hpa-single-cell-image-classification/train/{image_id}_red.png'
    
    # er is the endoplasmic reticulum
    er = f'/kaggle/input/hpa-single-cell-image-classification/train/{image_id}_yellow.png'
    
    # nu is the nuclei
    nu = f'/kaggle/input/hpa-single-cell-image-classification/train/{image_id}_blue.png'
    
    # high is the protein
    high = f'/kaggle/input/hpa-single-cell-image-classification/train/{image_id}_green.png'
    
    return [mt], [er], [nu], [high], [[mt], [er], [nu]],

def get_images(image_id):
    mt, er, nu, high, images = build_image_names(image_id=image_id)
    
    blue = plt.imread(nu[0])
    
    green = plt.imread(high[0])
    
    red = plt.imread(mt[0])
    
    yellow = plt.imread(er[0])
  
    return blue, green, red, yellow

I have been having problems with only a few images that end up crashing my notebooks when I compile with the entire dataset. One example is below. I debugged it back to how I am stacking the model in the code, but I do not know why it is having this issue. I can continue to by-pass these images, but this one is the smaller classes. Any thoughts on why this 3072 size image does not stack correctly and the subsequent one stacks as expected. 

In [ ]:
def normalization(array):
    a = (array - array.min())/(array.max()-array.min())
    return a

In [ ]:
dfpath = '../input/hpa-single-cell-image-classification/train.csv'
traindf = pd.read_csv(dfpath)
traindf['ID'] = traindf['ID'].str.strip()
names = np.array(traindf['ID'])
count = 0
for image_id in names:
    try:
        blue, green, red, yellow = get_images(image_id)
    except TypeError:
        print(f'Could not open: {image_id}')
    if blue.shape[0] == 3072 and count <= 10:
        print(image_id, blue.shape[0])
        count+=1
    else:
        break

As a suggestion from [Satwik](https://www.kaggle.com/p4rallax) I looked at the pixel values and have added some normalization and rescaling algorythms for both images. The first image improves with the normalization to 255, rescaling to RGB at 255,3,255 but normalization to 1 or as is are still difficult to decifer. The 2nd image is best as is or normalized to 1 but is not quite right normalized to 255 or rescaled to RGB at 255,3,255. 

# Am I missing something in how image '7930e80e-bbc8-11e8-b2bc-ac1f6b6435d0' is presented in comparison the image '0b252bcc-bbc8-11e8-b2bc-ac1f6b6435d0'? Revised to '5e22a522-bb99-11e8-b2b9-ac1f6b6435d0' which is also a 3072 image. 

In [ ]:
image_id = '7930e80e-bbc8-11e8-b2bc-ac1f6b6435d0' 

mt, er, nu, high, images = build_image_names(image_id=image_id)
    
# Protein model image
blue = plt.imread(nu[0])
green = plt.imread(high[0])
red = plt.imread(mt[0])
yellow = plt.imread(er[0])

RGB = np.stack((red,green,blue), axis = 2)
RGB_norm = np.stack((normalization(red), normalization(green), normalization(blue)), axis = 2)
RGB_255norm = np.stack((normalization(red)*255, normalization(green)*255, normalization(blue)*255), axis = 2)
RGB_Satwik = np.stack((normalization(red)*255, normalization(green)*3, normalization(blue)*255), axis = 2)
RYB = np.stack(((red-124)/(1/(.0167*255)) ,yellow,blue), axis = 2)

##NORMALIZE = {"mean": [124 / 255, NA, NA], "std": [1 / (0.0167 * 255), ] * 3}

plt.figure(figsize=(40,4)) 
print_cell(RGB, 'RGB stack', 1)
print_cell(RGB_norm, 'RGB stack - normalize to 1', 2)
print_cell(RGB_255norm, 'RGB stack - normalize to 255', 3)
print_cell(RGB_Satwik, 'RGB stack - 255,3,255 rescale', 4)
plt.tight_layout() 
plt.show()

plt.figure(figsize=(40,4)) 
print_cell(RYB, 'RYB stack from cell segmentator normalization', 1)
print_cell(red, 'red', 2)
print_cell(blue, 'blue', 3) 
print_cell(green, 'Green cells', 4) 
plt.tight_layout() 
plt.show()

print(f'stacked image has: {RGB.shape}, only blue / nuclei has: {blue.shape}')
print(f'Average pixel in blue: {blue.mean()}, red: {red.mean()}, green: {green.mean()}')
print(f'Max pixel in blue: {blue.max()}, red: {red.max()}, green: {green.max()}')


In [ ]:
image_id = '60b57878-bb99-11e8-b2b9-ac1f6b6435d0' 

mt, er, nu, high, images = build_image_names(image_id=image_id)
    
# Protein model image
blue = plt.imread(nu[0])
green = plt.imread(high[0])
red = plt.imread(mt[0])
yellow = plt.imread(er[0])

RGB = np.stack((red,green,blue), axis = 2)
RGB_norm = np.stack((normalization(red), normalization(green), normalization(blue)), axis = 2)
RGB_255norm = np.stack((normalization(red)*255, normalization(green)*255, normalization(blue)*255), axis = 2)
RGB_Satwik = np.stack((normalization(red)*255, normalization(green)*3, normalization(blue)*255), axis = 2)
RYB = np.stack(((red-124)/(1/(.0167*255)) ,yellow,blue), axis = 2)

##NORMALIZE = {"mean": [124 / 255, NA, NA], "std": [1 / (0.0167 * 255), ] * 3}

plt.figure(figsize=(40,4)) 
print_cell(RGB, 'RGB stack', 1)
print_cell(RGB_norm, 'RGB stack - normalize to 1', 2)
print_cell(RGB_255norm, 'RGB stack - normalize to 255', 3)
print_cell(RGB_Satwik, 'RGB stack - 255,3,255 rescale', 4)
plt.tight_layout() 
plt.show()

plt.figure(figsize=(40,4)) 
print_cell(RYB, 'RYB stack from cell segmentator normalization', 1)
print_cell(red, 'red', 2)
print_cell(blue, 'blue', 3) 
print_cell(green, 'Green cells', 4) 
plt.tight_layout() 
plt.show()

print(f'stacked image has: {RGB.shape}, only blue / nuclei has: {blue.shape}')
print(f'Average pixel in blue: {blue.mean()}, red: {red.mean()}, green: {green.mean()}')
print(f'Max pixel in blue: {blue.max()}, red: {red.max()}, green: {green.max()}')